In [1]:
import numpy as np
import requests
import json
import apikey
from requests.auth import HTTPBasicAuth
import matplotlib.pyplot as plt

import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# In this notebook
Testing how to use dataframe cleanly with less lines of code and making code more readable

# Get sensor locations from old API

In [2]:
# Set the API endpoint and request all devices
url = 'https://smartcampus.oulu.fi/manage/api/devices/listAll'
old_sensor_data = []

# Send the API request
response = requests.get(url)

# Check the response status code
if response.status_code == 200:
    print("200")
else:
    print("Error: API request failed with status code", response.status_code)
    
for line in response.iter_lines(decode_unicode=True):
    old_sensor_data.append(json.loads(line))

200


In [3]:
#First element on the list
print(old_sensor_data[0][0]['deviceId'])
print(old_sensor_data[0][0]['location']['coordinates'])

A81758FFFE046433
[65.05765, 25.46897]


In [4]:
#Dictionary with key and coordinates
sensor_list = {}
for sensor in old_sensor_data[0]:
    
    #Check for Nones in data
    if sensor['location'] == None:
        continue
        
    #without hyphens; ex. A81758FFFE046433
    old_id = sensor['deviceId']
    #with hyphens; ex. A8-17-58-FF-FE-04-64-33
    new_id = '-'.join(old_id[i:i+2] for i in range(0, len(old_id), 2))
    
    #Create dictionary {key new_id : value coordinates}
    sensor_list[new_id] = sensor['location']['coordinates']
    
    
    
sensor_list

{'A8-17-58-FF-FE-04-64-33': [65.05765, 25.46897],
 'A8-17-58-FF-FE-03-0F-6A': [65.05875, 25.46565],
 'A8-17-58-FF-FE-03-10-18': [65.05884, 25.46823],
 'A8-17-58-FF-FE-04-65-00': [65.06081, 25.46679],
 'A8-17-58-FF-FE-03-10-09': [65.05842, 25.46385],
 'A8-17-58-FF-FE-03-10-87': [65.0585, 25.46674],
 'A8-17-58-FF-FE-04-64-F8': [65.05994, 25.46657],
 'A8-17-58-FF-FE-03-0F-8B': [65.05961, 25.466],
 'A8-17-58-FF-FE-03-0F-68': [65.05963, 25.46852],
 'A8-17-58-FF-FE-03-10-0D': [65.05998, 25.46886],
 'A8-17-58-FF-FE-03-0F-93': [65.05965, 25.46385],
 'A8-17-58-FF-FE-04-64-AC': [65.05688, 25.46774],
 'A8-17-58-FF-FE-03-10-90': [65.05807, 25.46889],
 'A8-17-58-FF-FE-03-10-0F': [65.05785, 25.46883],
 'A8-17-58-FF-FE-04-64-B3': [65.05776, 25.46879],
 'A8-17-58-FF-FE-03-10-33': [65.05719, 25.46788],
 'A8-17-58-FF-FE-04-63-38': [65.05967, 25.4634],
 'A8-17-58-FF-FE-03-10-20': [65.05988, 25.46846],
 'A8-17-58-FF-FE-03-0F-D4': [65.06106, 25.46617],
 'A8-17-58-FF-FE-03-0F-B2': [65.06113, 25.46619],
 'A8

# Get sensor data from new API

In [5]:
#smart campus api key
headers = {
    'Content-type': 'application/json', 
    'Authorization': f'{apikey.APIKEY}',
}

In [6]:
# Set the API endpoint and request 15000 datapoints
url = 'https://query-api.rahtiapp.fi/events?limit=15000'

# Send the API request
response = requests.get(url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the sensor data
    sensor_data = response.iter_lines()
else:
    print("Error: API request failed with status code", response.status_code)

In [7]:
#Create list out of response
resp = []
for line in response.iter_lines(decode_unicode=True):
    resp.append(json.loads(line))

In [8]:
#Singe data point
resp[0][0]

{'time': '2023-03-31T00:00:02.470Z',
 'temperature': 12.1,
 'humidity': 21,
 'light': 1,
 'motion': 0,
 'co2': 474,
 'battery': 3.671,
 'deveui': 'a8-17-58-ff-fe-03-10-8d',
 'deveui_1': 'a8-17-58-ff-fe-03-10-8d'}

In [9]:
try:
    sensor_list[resp[0][0]['deveui'].upper()]
    print("Sensor found")
except KeyError:
    print("No locations for sensor")

Sensor found


In [46]:
#Create dataframe
df = pd.DataFrame(columns=['ID','time','temperature','humidity','light','motion','co2', 'coordinates'])

In [47]:
#iterate trough response list
for measurement in resp[0]:
    
    #add row to dataframe
    df.loc[len(df.index)]=[measurement['deveui'],       #id
                           datetime.datetime.fromisoformat(measurement['time']), #timestamp
                           measurement['temperature'],  #temperature
                           measurement['humidity'],     #humidity
                           measurement['light'],        #illuminance
                           measurement['motion'],       #motion
                           measurement['co2'],          #CO2
                           sensor_list[measurement['deveui'].upper()] #coordinates[lat, lng]
                          ]

In [12]:
for measurement in resp[0]:
    print(sensor_list[measurement['deveui'].upper()])

[65.05776, 25.46895]
[65.05715, 25.46785]
[65.06087, 25.46683]
[65.06107, 25.46675]
[65.05813404631877, 25.466177165508274]
[65.05849, 25.46598]
[65.05865, 25.4656]
[65.05785, 25.46718]
[65.05806, 25.46922]
[65.05882, 25.46564]
[65.06352, 25.4653]
[65.05916, 25.46565]
[65.05788437215459, 25.46949237585068]
[65.06146, 25.4672]
[65.06343, 25.46577]
[65.05891, 25.46798]
[65.05745, 25.46926]
[65.0611, 25.46679]
[65.05786, 25.46883]
[65.05864, 25.46656]
[65.05769, 25.4676]
[65.06043, 25.46667]
[65.05813404631877, 25.466177165508274]
[65.05973, 25.46816]
[65.06123, 25.46752]
[65.05775, 25.46891]
[65.05895, 25.46782]
[65.0606, 25.4667]
[65.05705, 25.46767]
[65.06131, 25.46819]
[65.06124, 25.46738]
[65.05792, 25.46972]
[65.05745, 25.46831]
[65.05782, 25.46833]
[65.05902, 25.46647]
[65.05725, 25.46777]
[65.05891, 25.46803]
[65.0586, 25.46656]
[65.05751, 25.46891]
[65.05813404631877, 25.466177165508274]
[65.05755, 25.46833]
[65.0585, 25.46674]
[65.05907, 25.46647]
[65.05991, 25.46633]
[65.06123,

In [48]:
df

,ID,time,temperature,humidity,light,motion,co2,coordinates
0,a8-17-58-ff-fe-03-10-8d,2023-03-31 00:00:02.470000+00:00,12.1,21,1,0,474,"[65.05776, 25.46895]"
1,a8-17-58-ff-fe-03-10-38,2023-03-31 00:00:04.332000+00:00,20.9,16,5,0,337,"[65.05715, 25.46785]"
2,a8-17-58-ff-fe-03-0f-bd,2023-03-31 00:00:08.907000+00:00,21.9,11,2,1,469,"[65.06087, 25.46683]"
3,a8-17-58-ff-fe-03-10-4e,2023-03-31 00:00:12.744000+00:00,20.9,13,191,0,386,"[65.06107, 25.46675]"
4,a8-17-58-ff-fe-04-63-b4,2023-03-31 00:00:18.510000+00:00,21.7,15,0,0,None,"[65.05813404631877, 25.466177165508274]"
...,...,...,...,...,...,...,...,...
8571,a8-17-58-ff-fe-04-63-3a,2023-03-31 07:44:28.850000+00:00,19,18,1727,75,None,"[65.05791, 25.46806]"
8572,a8-17-58-ff-fe-03-0f-87,2023-03-31 07:44:29.067000+00:00,20.7,16,391,8,570,"[65.05865, 25.46598]"
8573,a8-17-58-ff-fe-03-0f-a0,2023-03-31 07:44:33.534000+00:00,21.7,16,208,10,626,"[65.05854, 25.46551]"
8574,a8-17-58-ff-fe-03-10-07,2023-03-31 07:44:38.649000+00:00,20.6,15,349,11,484,"[65.06123, 25.46873]"


# Format data to fit HeatMap functions parameters

In [15]:
''' 
HeatMapWithTime()

Parameters
    ----------
    data: list of list of points of the form [lat, lng] or [lat, lng, weight]
        The points you want to plot. The outer list corresponds to the various time
        steps in sequential order. (weight is in (0, 1] range and defaults to 1 if
        not specified for a point)
    index: Index giving the label (or timestamp) of the elements of data. Should have
        the same length as data, or is replaced by a simple count if not specified.
    name : string, default None
        The name of the Layer, as it will appear in LayerControls.
'''

#Min-Max normalize column to get weights between [0, 1] range
column = "motion"
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
column = "light"
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())



In [18]:
motion_data = df[["coordinates", "motion", "time"]]

In [20]:
data = []
timestamps = []

for _, row in motion_data.iterrows():
    data.append([row['coordinates'][0],row['coordinates'][1],row['motion']])
    timestamps.append(row['time'])

data

[[65.05776, 25.46895, 0],
 [65.05715, 25.46785, 0],
 [65.06087, 25.46683, 1],
 [65.06107, 25.46675, 0],
 [65.05813404631877, 25.466177165508274, 0],
 [65.05849, 25.46598, 0],
 [65.05865, 25.4656, 0],
 [65.05785, 25.46718, 1],
 [65.05806, 25.46922, 0],
 [65.05882, 25.46564, 0],
 [65.06352, 25.4653, 0],
 [65.05916, 25.46565, 0],
 [65.05788437215459, 25.46949237585068, 0],
 [65.06146, 25.4672, 0],
 [65.06343, 25.46577, None],
 [65.05891, 25.46798, 0],
 [65.05745, 25.46926, 0],
 [65.0611, 25.46679, 2],
 [65.05786, 25.46883, 0],
 [65.05864, 25.46656, 0],
 [65.05769, 25.4676, 0],
 [65.06043, 25.46667, 2],
 [65.05813404631877, 25.466177165508274, 0],
 [65.05973, 25.46816, 0],
 [65.06123, 25.46752, 0],
 [65.05775, 25.46891, 0],
 [65.05895, 25.46782, 88],
 [65.0606, 25.4667, 0],
 [65.05705, 25.46767, 0],
 [65.06131, 25.46819, 1],
 [65.06124, 25.46738, 0],
 [65.05792, 25.46972, 39],
 [65.05745, 25.46831, 0],
 [65.05782, 25.46833, 0],
 [65.05902, 25.46647, 0],
 [65.05725, 25.46777, 0],
 [65.05891

# Change time to 24h format

In [30]:
timestamps[-1]

'2023-03-31T07:44:40.363Z'

In [56]:
from dateutil import parser

parser.parse(timestamps[-100])

datetime.datetime(2023, 3, 31, 7, 39, 10, 25000, tzinfo=tzutc())

In [52]:
test = datetime.datetime.fromisoformat(timestamps[-1])

In [53]:
test.time()

datetime.time(7, 44, 40, 363000)

In [51]:
datetime.time(7, 45, 0, 0) < test.time()

False